### 케라스에서 모델만들기

In [28]:
from tensorflow import keras
class SimpleDense(keras.layers.Layer) : #모든 케라스 층은 layers. 클래스 상속 
    def __init__(self, units, activation = None):
        super().__init__() #keras.layers = 부모함수 호출 
        self.units= units
        self.activation = activation
        
    def build(self, input_shape): #build , call 이름도 그대로 정의
        input_dim = input_shape[-1]
        self.W=self.add_weight(shape=(input_dim, self.units),
                              initializer = 'random_normal')
        self.b= self.add_weight(shape=(self.units,), 
                               initializer= 'zeros')
        
    def call (self,inputs): #call()에서 정방향 패스 계산 구현
        y= tf.matmul(inputs, self.W)+self.b
        if self.activation is not None:
            y= self.activation(y)
        return y

In [29]:
#인스턴스 생성하면 텐서를 입력받는 함수처럼 사용가능 
import tensorflow as tf
my_dense= SimpleDense(units=32, activation = tf.nn.relu)

In [30]:
input_tensor= tf.ones(shape=(2,784))

In [31]:
output_tensor= my_dense(input_tensor)
output_tensor.shape

TensorShape([2, 32])

In [ ]:
model= models.Sequential([layers.Dense(32, activation='relu'),
                         layers.Dense(32)])

In [36]:
model=keras.Sequential()
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax' ))

NameError: name 'layers' is not defined

### 함수형api

In [37]:
#2개의 dense 층을 가진 함수형 모델 
from tensorflow.keras import layers
inputs= keras.Input(shape=(3,),name='my_input' )
features=layers.Dense(64, activation='relu')(inputs)
outputs= layers.Dense(10, activation='softmax')(features)
model =keras.Model(inputs=inputs, outputs=outputs )

In [38]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 64)                256       
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


### 다중입력 , 다중출력모델

In [4]:
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras import layers
import keras
vocab_size= 10000
num_tags= 100
num_departments=4

title= Input(shape=(vocab_size,),name='title' ) #입력 정의
text_body= Input(shape=(vocab_size,),name='text_body' )
tags= Input(shape=(num_tags,),name='tags' )

features=layers.Concatenate()([title,text_body,tags]) #입력특성을 하나의 텐서 feature로 연결 
features=layers.Dense(64, activation='relu')(features) #중간층으로 재결합

priority=layers.Dense(1, activation='sigmoid', name='priority')(features)  #출력 정의 
department=layers.Dense(num_departments, activation='softmax', name='department')(features)
model=keras.Model(inputs=[title, text_body, tags], #입출력 지정
                 outputs=[priority, department])

In [5]:
#입력과 타겟 배열 리스트를 전달하여 모델 훈련 
import numpy as np
num_samples=1280

title_data= np.random.randint(0,2,size=(num_samples, vocab_size))
text_body_data= np.random.randint(0,2,size=(num_samples, vocab_size))
tags_data= np.random.randint(0,2,size=(num_samples, num_tags))  #dummy 입력 데이터 

priority_data= np.random.random(size=(num_samples,1))
department_data= np.random.randint(0,2,size=(num_samples, num_departments)) #더미 타겟 

model.compile(optimizer ='rmsprop', 
             loss =['mean_squared_error','categorical_crossentropy'],
             metrics=[['mean_absolute_error'],['accuracy']])

model.fit([title_data,text_body_data, tags_data], #데이터 리스트는 model 클래스에 전달한 순서와 같아야한다. 
          [priority_data,department_data],
                   epochs=1)

model.evaluate([title_data,text_body_data, tags_data],
          [priority_data,department_data])
prioridy_preds, department_preds = model.predict([title_data, text_body_data, tags_data])

40/40 [==============================] - 0s 3ms/step


In [84]:
!pip install graphviz

                                              0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 47.0/47.0 kB ? eta 0:00:00


In [103]:
!pip install pydot

In [6]:
#층 연결 구조 확인
import graphviz
import pydot
keras.utils.plot_model(model, 'ticket_classifier.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


### 합성곱

In [45]:
from tensorflow import keras

from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten

In [46]:
#간단한 컨브넷 
inputs=Input(shape=(28,28,1))
x= Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x= MaxPooling2D(pool_size=2)(x)
x= Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x= MaxPooling2D(pool_size=2)(x)
x= Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x= Flatten()(x)
outputs=Dense(units=10, activation='softmax')(x)
model=keras.Model(inputs=inputs, outputs=outputs)



In [47]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         7385

In [53]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import mnist

(train_images, train_labels),(test_images, test_labels)=mnist.load_data()
train_images= train_images.reshape((60000, 28,28,1)) #4차원, 전에는 3차원으로 (60000,28*28)
train_images=train_images.astype('float32')/255
test_images= test_images.reshape((10000,28,28,1))
test_images = test_images.astype ('float32')/ 255

model.compile(optimizer ='rmsprop', 
             loss ='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

#컨브넷 평가

test_loss, test_acc= model.evaluate (test_images, test_labels)
print(f'테스트 정확도:{test_acc:.3f}') #테스트 정확도:0.993

Epoch 1/5
938/938 [==============================] - 15s 15ms/step - loss: 0.0139 - accuracy: 0.9958
Epoch 2/5
938/938 [==============================] - 14s 15ms/step - loss: 0.0102 - accuracy: 0.9969
Epoch 3/5
938/938 [==============================] - 14s 15ms/step - loss: 0.0085 - accuracy: 0.9975
Epoch 4/5
938/938 [==============================] - 15s 16ms/step - loss: 0.0069 - accuracy: 0.9981
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.0325 - accuracy: 0.9925
테스트 정확도:0.993


### 고양이 개 이미지 분류 p.296 

In [55]:
!pip install gdown

In [56]:
import gdown 
url = "https://drive.google.com/uc?id=1ipzN9okFFT3oieklsrfr6iUHzjPAbT9i"
gdown.download(url, "cats_and_dogs.tar") 

Downloading...
From: https://drive.google.com/uc?id=1ipzN9okFFT3oieklsrfr6iUHzjPAbT9i
To: C:\Users\user\cats_and_dogs.tar
100%|█████████████████████████████████████████████████████████████████████████████| 94.0M/94.0M [00:02<00:00, 46.8MB/s]


'cats_and_dogs.tar'

In [62]:
!tar -xvf cats_and_dogs.tar  
#압축풀기 

x cats_and_dogs/
x cats_and_dogs/test/
x cats_and_dogs/train/
x cats_and_dogs/validation/
x cats_and_dogs/test/cats/
x cats_and_dogs/test/dogs/
x cats_and_dogs/train/cats/
x cats_and_dogs/train/dogs/
x cats_and_dogs/validation/cats/
x cats_and_dogs/validation/dogs/
x cats_and_dogs/test/cats/cat.1500.jpg
x cats_and_dogs/test/cats/cat.1501.jpg
x cats_and_dogs/test/cats/cat.1502.jpg
x cats_and_dogs/test/cats/cat.1503.jpg
x cats_and_dogs/test/cats/cat.1504.jpg
x cats_and_dogs/test/cats/cat.1505.jpg
x cats_and_dogs/test/cats/cat.1506.jpg
x cats_and_dogs/test/cats/cat.1507.jpg
x cats_and_dogs/test/cats/cat.1508.jpg
x cats_and_dogs/test/cats/cat.1509.jpg
x cats_and_dogs/test/cats/cat.1510.jpg
x cats_and_dogs/test/cats/cat.1511.jpg
x cats_and_dogs/test/cats/cat.1512.jpg
x cats_and_dogs/test/cats/cat.1513.jpg
x cats_and_dogs/test/cats/cat.1514.jpg
x cats_and_dogs/test/cats/cat.1515.jpg
x cats_and_dogs/test/cats/cat.1516.jpg
x cats_and_dogs/test/cats/cat.1517.jpg
x cats_and_dogs/test/cats/cat.15

In [63]:

train_dir = r'cats_and_dogs\train'
validation_dir = r'cats_and_dogs\validation'
test_dir = r'cats_and_dogs\test'



## 폴더별 데이터 분류

import os
# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir, 'dogs')

# 검증용 고양이 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir, 'cats')

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir, 'dogs')


print(train_cats_dir)

print('훈련용 고양이 이미지 전체 개수:', len(os.listdir(train_cats_dir)))
print('훈련용 강아지 이미지 전체 개수:', len(os.listdir(train_dogs_dir)))
print('검증용 고양이 이미지 전체 개수:', len(os.listdir(validation_cats_dir)))
print('검증용 강아지 이미지 전체 개수:', len(os.listdir(validation_dogs_dir)))
print('테스트용 고양이 이미지 전체 개수:', len(os.listdir(test_cats_dir)))
print('테스트용 강아지 이미지 전체 개수:', len(os.listdir(test_dogs_dir))) 

cats_and_dogs\train\cats
훈련용 고양이 이미지 전체 개수: 1000
훈련용 강아지 이미지 전체 개수: 1000
검증용 고양이 이미지 전체 개수: 500
검증용 강아지 이미지 전체 개수: 500
테스트용 고양이 이미지 전체 개수: 500
테스트용 강아지 이미지 전체 개수: 500


In [60]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 144F-541F

 C:\Users\user 디렉터리

2023-06-20  오후 03:45    <DIR>          .
2023-05-01  오전 08:30    <DIR>          ..
2023-06-20  오후 03:42    <DIR>          .cache
2023-04-28  오전 09:02    <DIR>          .conda
2023-03-24  오전 11:12                25 .condarc
2023-03-24  오전 11:12    <DIR>          .continuum
2023-03-09  오후 12:09    <DIR>          .idlerc
2023-06-20  오전 09:48    <DIR>          .ipynb_checkpoints
2023-03-09  오후 12:37    <DIR>          .ipython
2023-03-10  오전 09:12    <DIR>          .jupyter
2023-06-12  오후 02:14    <DIR>          .keras
2023-03-09  오후 03:51    <DIR>          .matplotlib
2023-02-28  오전 09:07    <DIR>          .ms-ad
2023-04-06  오전 09:20    <DIR>          .pydataset
2023-03-24  오전 11:43    <DIR>          .spyder-py3
2023-05-11  오후 03:46    <DIR>          .surprise_data
2023-06-12  오후 03:21            24,057 0612 keras.ipynb
2023-06-14  오후 12:30            10,911 0613.ipynb
2023-06-15  오전 10:30           137,776 0614.ipynb
2023-

In [64]:
import keras

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(       
        train_dir,               # 타깃 디렉터리
        target_size=(150, 150),  # 모든 이미지를 150 × 150 크기로 바꿉니다
        batch_size=20,           #  20개씩 배치로 생산        
        class_mode='binary')     # 이진 레이블. 만약 다중클래스라면 'categorical'

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary') 

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [66]:
## 모델 구성

from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size= (3,3), activation="relu", input_shape=(150, 150, 3)))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters=64, kernel_size= (3,3), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters=128, kernel_size= (3,3), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters=128, kernel_size= (3,3), activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(units=512, activation= 'relu' ))
model.add(layers.Dense(units=1, activation= 'sigmoid'))

## compile

model.compile(loss='binary_crossentropy',  # 이진이므로 loss는 binary_crossentropy
              optimizer='RMSprop',
              metrics=['accuracy'])

model.summary() 

model.fit(train_generator, epochs=5) 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 17, 17, 128)     

In [90]:
from tensorflow import keras
from tensorflow.keras import layers

model=keras.Sequential([layers.Dense(64, activation='relu'),
                       layers.Dense(10, activation='softmax')])

In [95]:
model= keras.Sequential(name='my_example_model')
model.add(layers.Dense(64, activation='relu',name='my_layer1'))
model.add(layers.Dense(10, activation='softmax',name='my_layer2'))
model.build((None,3))
model.summary()

Model: "my_example_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_layer1 (Dense)           (None, 64)                256       
                                                                 
 my_layer2 (Dense)           (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.build(input_shape=(None,3)) #build()메서드 호출 : 크기가 (3, )인 샘플을 기대 , none 어떤 배치크기도 가능 
model.weights

[<tf.Variable 'dense_10/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[ 0.10901624,  0.09896332, -0.18912205,  0.23180723, -0.28801972,
          0.24204761, -0.15488571, -0.16521825, -0.18895611, -0.14082542,
         -0.24596556,  0.03856736, -0.24424458,  0.00794795,  0.01069099,
          0.20520341,  0.17768556,  0.06811082, -0.02629679,  0.20601714,
         -0.22682996,  0.23266625, -0.19189847, -0.06801015,  0.24556738,
         -0.02313042,  0.10874626,  0.21473134, -0.2284101 , -0.08074695,
          0.03292257, -0.13132378, -0.280236  ,  0.18332237, -0.2714559 ,
         -0.05330388, -0.06053187,  0.18134448,  0.24147385, -0.29275855,
         -0.02316037, -0.06968254,  0.28465682, -0.01090261,  0.17993459,
         -0.29440567, -0.26203534, -0.06061949, -0.28494608,  0.07243833,
          0.20054287,  0.07624951,  0.1420264 , -0.03784409, -0.1667916 ,
         -0.00122318,  0.03949365, -0.12838407, -0.00162458,  0.17727304,
          0.09301722,  0.03119087, -0.2546

In [96]:
### 함수형 API

In [101]:
inputs= Input(shape=(3,),name='my_input' )# inputs.shape #이런 객체 심볼릭 텐서 , 배치크기 NONE, 샘플크기 (3,)
features=layers.Dense(64, activation='relu')(inputs) 
outputs=layers.Dense(10, activation='softmax')(features)
model=keras.Model(inputs=inputs, outputs=outputs)
model.summary()




Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_18 (Dense)            (None, 64)                256       
                                                                 
 dense_19 (Dense)            (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________
